### Import modules
___


In [ ]:
# Import modules
import os
from pathlib import Path

### Setup params & configs
___

In [ ]:
# Setup configs for Azure Gen 2 Blob Storage
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "client_id",
    "fs.azure.account.oauth2.client.secret": "client_secret",
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"
    }

In [ ]:
# Check if the mount point already exists
mount_point = "/mnt/paris-olympic-data"

# Unmount if the mount point exists
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)
    print(f"Unmounted: {mount_point}")

# Mount the Azure Blob Storage (as if it were a local file system)
dbutils.fs.mount(
    source = "abfss://paris-olympic-data@parisdata.dfs.core.windows.net", # contrainer@storageacc
    mount_point = mount_point,
    extra_configs = configs
    )

print(f"Mounted: {mount_point}")

Mounted: /mnt/paris-olympic-data


In [ ]:
%fs
ls "/mnt/paris-olympic-data"

path,name,size,modificationTime
dbfs:/mnt/paris-olympic-data/data-raw/,data-raw/,0,1725478007000
dbfs:/mnt/paris-olympic-data/data-transformed/,data-transformed/,0,1725478017000


In [ ]:
# Initialize Spark session
spark

### Load data
___

In [ ]:
# Read the data into Spark DataFrames
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/paris-olympic-data/data-raw/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/paris-olympic-data/data-raw/coaches.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/paris-olympic-data/data-raw/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/paris-olympic-data/data-raw/teams.csv")

### View data
___

In [ ]:
# Display data
# Limit to 10x10 so it prints nicely
athletes[athletes.columns[0:10]].show(n=10)

+-------+-------+--------------------+---------------+--------------------+------+--------+------------+--------+------------+
|   code|current|                name|     name_short|             name_tv|gender|function|country_code| country|country_long|
+-------+-------+--------------------+---------------+--------------------+------+--------+------------+--------+------------+
|1532872|   true|    ALEKSANYAN Artur|   ALEKSANYAN A|    Artur ALEKSANYAN|  Male| Athlete|         ARM| Armenia|     Armenia|
|1532873|   true|      AMOYAN Malkhas|       AMOYAN M|      Malkhas AMOYAN|  Male| Athlete|         ARM| Armenia|     Armenia|
|1532874|   true|     GALSTYAN Slavik|     GALSTYAN S|     Slavik GALSTYAN|  Male| Athlete|         ARM| Armenia|     Armenia|
|1532944|   true|   HARUTYUNYAN Arsen|  HARUTYUNYAN A|   Arsen HARUTYUNYAN|  Male| Athlete|         ARM| Armenia|     Armenia|
|1532945|   true|     TEVANYAN Vazgen|     TEVANYAN V|     Vazgen TEVANYAN|  Male| Athlete|         ARM| Armeni

In [ ]:
# Print schema
athletes.printSchema()

root
 |-- code: integer (nullable = true)
 |-- current: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- name_short: string (nullable = true)
 |-- name_tv: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- function: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- nationality_long: string (nullable = true)
 |-- nationality_code: string (nullable = true)
 |-- height: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- disciplines: string (nullable = true)
 |-- events: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- birth_place: string (nullable = true)
 |-- birth_country: string (nullable = true)
 |-- residence_place: string (nullable = true)
 |-- residence_country: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- hobbies: string (nullable = t

In [ ]:
coaches[coaches.columns[0:10]].show(n=10)

+-------+-------+-------------------+------+---------------+--------+------------+---------+--------------------+-----------------+
|   code|current|               name|gender|       function|category|country_code|  country|        country_long|      disciplines|
+-------+-------+-------------------+------+---------------+--------+------------+---------+--------------------+-----------------+
|1533246|   true|     PEDRERO Ofelia|Female|          Coach|       C|         MEX|   Mexico|              Mexico|Artistic Swimming|
|1535775|   true|   RADHI SHENAISHIL|  Male|     Head Coach|       C|         IRQ|     Iraq|                Iraq|         Football|
|1536055|   true|AFLAKIKHAMSEH Majid|  Male|          Coach|       C|         IRI|  IR Iran|Islamic Republic ...|        Taekwondo|
|1536059|   true|    YOUSEFY Mehrdad|  Male|          Coach|       C|         IRI|  IR Iran|Islamic Republic ...|        Taekwondo|
|1536060|   true|       MADDAH Minoo|Female|          Coach|       C|       

In [ ]:
coaches.printSchema()

root
 |-- code: integer (nullable = true)
 |-- current: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- function: string (nullable = true)
 |-- category: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)
 |-- disciplines: string (nullable = true)
 |-- events: string (nullable = true)
 |-- birth_date: date (nullable = true)



In [ ]:
medals[medals.columns[0:10]].show(n=10)

+------------+----------+----------+---------------+------+------------+--------------------+----------+--------------------+-----------------+
|  medal_type|medal_code|medal_date|           name|gender|  discipline|               event|event_type|           url_event|             code|
+------------+----------+----------+---------------+------+------------+--------------------+----------+--------------------+-----------------+
|  Gold Medal|       1.0|2024-07-27|Remco EVENEPOEL|     M|Cycling Road|Men's Individual ...|       ATH|/en/paris-2024/re...|          1903136|
|Silver Medal|       2.0|2024-07-27|  Filippo GANNA|     M|Cycling Road|Men's Individual ...|       ATH|/en/paris-2024/re...|          1923520|
|Bronze Medal|       3.0|2024-07-27|  Wout van AERT|     M|Cycling Road|Men's Individual ...|       ATH|/en/paris-2024/re...|          1903147|
|  Gold Medal|       1.0|2024-07-27|    Grace BROWN|     W|Cycling Road|Women's Individua...|       ATH|/en/paris-2024/re...|          1

In [ ]:
medals.printSchema()

root
 |-- medal_type: string (nullable = true)
 |-- medal_code: double (nullable = true)
 |-- medal_date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- event: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- url_event: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)



In [ ]:
teams[teams.columns[0:10]].show(n=10)

+-----------------+-------+--------------------+-----------+------------+-------------+--------------------+----------+----------------+----------+
|             code|current|                team|team_gender|country_code|      country|        country_long|discipline|disciplines_code|    events|
+-----------------+-------+--------------------+-----------+------------+-------------+--------------------+----------+----------------+----------+
|ARCMTEAM3---CHN01|   true|People's Republic...|          M|         CHN|        China|People's Republic...|   Archery|             ARC|Men's Team|
|ARCMTEAM3---COL01|   true|            Colombia|          M|         COL|     Colombia|            Colombia|   Archery|             ARC|Men's Team|
|ARCMTEAM3---FRA01|   true|              France|          M|         FRA|       France|              France|   Archery|             ARC|Men's Team|
|ARCMTEAM3---GBR01|   true|       Great Britain|          M|         GBR|Great Britain|       Great Britain|   A

In [ ]:
teams.printSchema()

root
 |-- code: string (nullable = true)
 |-- current: boolean (nullable = true)
 |-- team: string (nullable = true)
 |-- team_gender: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- disciplines_code: string (nullable = true)
 |-- events: string (nullable = true)
 |-- athletes: string (nullable = true)
 |-- coaches: string (nullable = true)
 |-- athletes_codes: string (nullable = true)
 |-- num_athletes: string (nullable = true)
 |-- coaches_codes: string (nullable = true)
 |-- num_coaches: string (nullable = true)



### Transform data
___
1) Change data type
2) Partition, column selection & write files

**1. Change data type** </br>
Medal_type is loaded in as a double .. check to see if this can be converted to an int.

In [ ]:
# Values can only be whole numbers
medals.toPandas().medal_code.value_counts()

3.0    384
2.0    330
1.0    329
Name: medal_code, dtype: int64

In [ ]:
# Change data type
medals = medals.withColumn("medal_code", medals["medal_code"].cast("int"))

In [ ]:
# Check that medal_code is now an integer
medals[medals.columns[0:10]].show(n=10)

+------------+----------+----------+---------------+------+------------+--------------------+----------+--------------------+-----------------+
|  medal_type|medal_code|medal_date|           name|gender|  discipline|               event|event_type|           url_event|             code|
+------------+----------+----------+---------------+------+------------+--------------------+----------+--------------------+-----------------+
|  Gold Medal|         1|2024-07-27|Remco EVENEPOEL|     M|Cycling Road|Men's Individual ...|       ATH|/en/paris-2024/re...|          1903136|
|Silver Medal|         2|2024-07-27|  Filippo GANNA|     M|Cycling Road|Men's Individual ...|       ATH|/en/paris-2024/re...|          1923520|
|Bronze Medal|         3|2024-07-27|  Wout van AERT|     M|Cycling Road|Men's Individual ...|       ATH|/en/paris-2024/re...|          1903147|
|  Gold Medal|         1|2024-07-27|    Grace BROWN|     W|Cycling Road|Women's Individua...|       ATH|/en/paris-2024/re...|          1

**2. Partition, column selection & write files** </br>
Partition files into 1 partition (for ease) and write to data-transformed directory

In [ ]:
athletes[athletes.columns[0:10]].repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/paris-olympic-data/data-transformed/athletes")
coaches[coaches.columns[0:10]].repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/paris-olympic-data/data-transformed/coaches")
medals[medals.columns[0:10]].repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/paris-olympic-data/data-transformed/medals")
teams[teams.columns[0:10]].repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/paris-olympic-data/data-transformed/teams")